In [34]:
import twint
import nest_asyncio
import tqdm
import datetime
nest_asyncio.apply()
DATA_FILE = "data_test/"

In [35]:
search_terms = []
with open('config/search-terms.txt') as f:
    lines = f.read().splitlines()
search_terms = lines
print(search_terms)

['abu_sayyaf', 'afghanistan', 'agro', 'al-qaeda', 'al-qaeda_in_the_arabian_peninsula', 'al-qaeda_in_the_islamic_maghreb', 'al-shabaab', 'ammonium_nitrate', 'attack', 'biological_weapon', 'car_bomb', 'chemical_weapon', 'conventional_weapon', 'dirty_bomb', 'eco-terrorism', 'environmental_terrorism', 'euskadi_ta_askatasuna', 'extremism', 'farc', 'fundamentalism', 'hamas', 'hezbollah', 'improvised_explosive_device', 'iran', 'iraq', 'irish_republican_army', 'islamist', 'jihad', 'nationalism', 'nigeria', 'nuclear', 'nuclear_enrichment', 'pakistan', 'palestine_liberation_front', 'pirates', 'plo', 'political_radicalism', 'recruitment', 'somalia', 'suicide_attack', 'suicide_bomber', 'taliban', 'tamil_tigers', 'tehrik-i-taliban_pakistan', 'terror', 'terrorism', 'weapons-grade', 'yemen']


In [36]:
short_terms = ['abu_sayyaf', 'al-qaeda', 'al-qaeda_in_the_arabian_peninsula', 'al-qaeda_in_the_islamic_maghreb', 'al-shabaab', 'ammonium_nitrate', 'biological_weapon', 'car_bomb', 'chemical_weapon', 'conventional_weapon', 'dirty_bomb', 'eco-terrorism', 'environmental_terrorism', 'euskadi_ta_askatasuna', 'extremism', 'farc', 'fundamentalism', 'hezbollah', 'improvised_explosive_device', 'irish_republican_army', 'jihad', 'nuclear_enrichment', 'palestine_liberation_front', 'political_radicalism', 'suicide_attack', 'suicide_bomber', 'taliban', 'tamil_tigers', 'tehrik-i-taliban_pakistan', 'weapons-grade']

In [37]:
long_terms = ['afghanistan', 'hamas', 'iran', 'iraq', 'islamist', 'nationalism', 'nigeria', 'pakistan', 'somalia', 'terrorism', 'yemen']

In [38]:
black_list = ['agro', 'attack', 'nuclear', 'pirates', 'plo', 'recruitment', 'terror']

In [39]:
concatenated = short_terms+long_terms+black_list
for elem in search_terms:
    if elem not in concatenated :
        print("The two lists differ")

In [40]:
dates = []
for y in ['2012', '2013', '2014'] :
    for m in range(1,12 + 1):
        dates.append(f"{y}-{m:02d}-01")
print(dates)

['2012-01-01', '2012-02-01', '2012-03-01', '2012-04-01', '2012-05-01', '2012-06-01', '2012-07-01', '2012-08-01', '2012-09-01', '2012-10-01', '2012-11-01', '2012-12-01', '2013-01-01', '2013-02-01', '2013-03-01', '2013-04-01', '2013-05-01', '2013-06-01', '2013-07-01', '2013-08-01', '2013-09-01', '2013-10-01', '2013-11-01', '2013-12-01', '2014-01-01', '2014-02-01', '2014-03-01', '2014-04-01', '2014-05-01', '2014-06-01', '2014-07-01', '2014-08-01', '2014-09-01', '2014-10-01', '2014-11-01', '2014-12-01']


In [41]:
dummy_terms = search_terms[:2]
dummy_dates = dates[:2]
print(dummy_terms)
print(dummy_dates)

['abu_sayyaf', 'afghanistan']
['2012-01-01', '2012-02-01']


In [42]:
import threading
import multiprocessing
from time import sleep
from joblib import Parallel, delayed

NB_PROC = 12
NB_THREAD_P_PROC = 20

In [43]:
def process_run(task_queue):
    for i in range(NB_THREAD_P_PROC):
        thread = Thread(target = thread_run, args=(task_queue))
        thread.start()

def thread_run(task_queue):
    continue_ = True
    while continue_:
        task = task_queue.get()
        if task == None: #TODO change
            tadk_queue.put(None)
            continue_ = False
            break
        term = task[0]
        since = task[1]
        until = task[2]
        print("Beginning scrapping ", term, " from ", since, " until ", until)
        Scrap(term, since, until)
        print("Finished scrapping ", term, " from ", since, " until ", until)
        if (int(until[-2:])>25):
            print("##################### Finished term ", term)
    print("No more element in the queue, terminating")

def Scrap(term, since, until):
    print(term, ' from ', since, ' to ', until)
    c = twint.Config()
    c.Search = term
    c.Since = since
    c.Until = until
    #c.Until = "2012-06-01"
    c.Output = DATA_FILE+term+'/'+since+'-'+term+".csv"
    c.Hide_output = True
    c.Store_csv = True
    twint.run.Search(c)
    
Scrap('pineapple', '2012-02-02', '2012-02-03')

pineapple  from  2012-02-02  to  2012-02-03


AttributeError: module 'twint_lib' has no attribute 'Config'

In [ ]:
work = [[i, dummy_dates[j], dummy_dates[j+1]] for j in range(len(dummy_dates)-1) for i in dummy_terms]
work = sorted(work, key= lambda elem: elem[0])
print(work)
task_queue = multiprocessing.Queue()
for w in work:
    task_queue.put(w)
for i in range(NB_PROC):
    task_queue.put(None)

In [ ]:
if __name__ == '__main__':
    task_queue = multiprocessing.Queue()
    work = [['abu_sayyaf', '2012-01-01', '2012-02-01'], ['afghanistan', '2012-01-01', '2012-02-01']]
    for w in work:
        task_queue.put(w)
    for i in range(NB_PROC):
        task_queue.put(None)
    num_cores = multiprocessing.cpu_count()
    print(num_cores)
    the_pool = multiprocessing.Pool(NB_PROC, process_run, (task_queue,))
    #results = Parallel(n_jobs=NB_PROC)(delayed(process_run)(task_queue) for i in range(NB_PROC))
    sleep(10)

In [ ]:
%run multiprocessing-script.py

In [ ]:
print('''
Time taken to scrap 
abu_sayyaf : 7 min 44 sec, 11.8 MB
afghanistan : 

''')